In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/multiple_hosp_patients/train_test/train_tuning.csv")
validation_df = pd.read_csv("/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/multiple_hosp_patients/train_test/validation_tuning.csv")
full_train_df = pd.read_csv("/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/multiple_hosp_patients/train_test/train_full.csv")

In [3]:
assert train_df.shape[0] + validation_df.shape[0] == full_train_df.shape[0], "Train and validation sets do not sum up to full training set."

In [4]:
train_event_ids = set(train_df['HADM_ID'].tolist())
validation_event_ids = set(validation_df['HADM_ID'].tolist())
full_train_event_ids = set(full_train_df['HADM_ID'].tolist())

subject_ids_in_train = set(train_df['SUBJECT_ID'].tolist())
subject_ids_in_validation = set(validation_df['SUBJECT_ID'].tolist())
subject_ids_in_full_train = set(full_train_df['SUBJECT_ID'].tolist())

assert subject_ids_in_train.isdisjoint(subject_ids_in_validation), "Train and validation sets have overlapping SUBJECT_IDs."
assert subject_ids_in_train.union(subject_ids_in_validation) == subject_ids_in_full_train, "Union of train and validation sets does not equal full training set."
assert train_event_ids.isdisjoint(validation_event_ids), "Train and validation sets have overlapping HADM_IDs."
assert train_event_ids.union(validation_event_ids) == full_train_event_ids, "Union of train and validation event IDs does not equal full training event IDs."

In [5]:
print("Num of samples in train set:", train_df.shape[0])
print("Num of samples in validation set:", validation_df.shape[0])
print("Num of samples in full train set:", full_train_df.shape[0])

Num of samples in train set: 3888
Num of samples in validation set: 904
Num of samples in full train set: 4792


In [6]:
import torch

base_path = "/workspaces/msc-thesis-recurrent-health-modeling/_models/mimic/deep_learning/attention_pooling/multiple_hosp_patients/"
full_train_dataset = torch.load(base_path + "train_final_dataset.pt", weights_only=False)
test_dataset = torch.load(base_path + "test_dataset.pt", weights_only=False)

In [7]:
full_train_dataset.reverse_chronological_order

False

In [8]:
test_dataset.reverse_chronological_order

False

In [9]:
val_final_dataset = torch.load(base_path + "validation_final_dataset.pt", weights_only=False)
val_final_dataset.reverse_chronological_order

False

In [10]:
train_fine_tune_dataset = torch.load(base_path + "train_tuning_dataset.pt", weights_only=False)
train_fine_tune_dataset.reverse_chronological_order

False